In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2.tools.analysis as analysisTools
import timeit
from matplotlib import rc
rc('text', usetex=True)

In [ ]:
# Main parameters
numparticles = 2
boxsize = 6 #10 #8 #6 #5 
D = 1.0
Drot = 1.0
numTrajectories = 1000

In [ ]:
# Color definitions
c1 = '#ff3300' # for benchmark
c2 = '#0099ff' # for MSM/RD
# Alpha transparency definition
a1 = 0.2
a2 = 0.7

## Get unbound2bound FPT data of bechmark and MSMRD from files

In [ ]:
# Load benchmark FPT data computed by script
numTrajs = 10000
filename1 = '../../data/dimer/first_passage_times/patchyDimerFPTs_2bound_trajs' + \
str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
fptsA = []
fptsB = []
with open(filename1) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        if state == 'A':
            fptsA.append(time)
        elif state == 'B':
            fptsB.append(time)

In [ ]:
# Load FPT data obtained with MSMRD integrator
lagtime = 150
numTrajs = 10000
filename = '../../data/dimer/first_passage_times/MSMRDpatchyDimerFPTs_2bound_trajs' + \
str(numTrajs) + '_lagt' + str(lagtime) +  '_boxsize' + str(boxsize) + '.xyz'
MSMRDfptsA = []
MSMRDfptsB = []
with open(filename) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        if state == 'A':
            MSMRDfptsA.append(time)
        elif state == 'B':
            MSMRDfptsB.append(time)

## Plots of FPT data from benchmark and MSMRD simulations

In [ ]:
# Compuate mean and standard deviation of bootstrapped samples of benchmark
calculateRates = True
numBootsrapSamples = 2000
mfptA, fptAstd = analysisTools.bootstrapping(fptsA, numBootsrapSamples)
mfptB, fptBstd = analysisTools.bootstrapping(fptsB, numBootsrapSamples)
koffA = 1.0/np.array(fptsA)
koffB = 1.0/np.array(fptsB)
print("Raw MFPTs (A and B): ", np.array(fptsA).mean(), np.array(fptsB).mean() )
print("Bootstrapping mean and std (A): ", mfptA, fptAstd)
print("Bootstrapping mean and std (B): ", mfptB, fptBstd)
# Compuate mean and standard deviation of bootstrapped samples of MSMRD simulation
MSMRDmfptA, MSMRDfptAstd = analysisTools.bootstrapping(MSMRDfptsA, numBootsrapSamples)
MSMRDmfptB, MSMRDfptBstd = analysisTools.bootstrapping(MSMRDfptsB, numBootsrapSamples)
print("Raw MSMRD MFPTs (A and B): ",np.array(MSMRDfptsA).mean(), np.array(MSMRDfptsB).mean())
print("MSMRD bootstrapping mean and std (A): ", MSMRDmfptA, MSMRDfptAstd)
print("MSMRD bootstrapping mean and std (B): ", MSMRDmfptB, MSMRDfptBstd)
print("Trajectory lengths")
print("FPTs A", len(fptsA), "FPTs MSMRD-A", len(MSMRDfptsA))
print("FPTs B", len(fptsB), "FPTs MSMRD-B", len(MSMRDfptsB))
ratioFPT = float(len(fptsA))/float(len(fptsB))
ratioMSMRDFPT = float(len(MSMRDfptsA))/float(len(MSMRDfptsB))
print("Ratios:", ratioFPT, ratioMSMRDFPT )
if calculateRates:
    konA = 1.0/np.array(fptsA)
    konB = 1.0/np.array(fptsB)
    MSMRDkonA = 1.0/np.array(MSMRDfptsA)
    MSMRDkonB = 1.0/np.array(MSMRDfptsB)
    mkonA, konAstd = analysisTools.bootstrapping(konA, numBootsrapSamples)
    mkonB, konBstd = analysisTools.bootstrapping(konB, numBootsrapSamples)
    MSMRDmkonA, MSMRDkonAstd = analysisTools.bootstrapping(MSMRDkonA, numBootsrapSamples)
    MSMRDmkonB, MSMRDkonBstd = analysisTools.bootstrapping(MSMRDkonB, numBootsrapSamples)
    print("On rates:")
    print("Bootstrapping mean and std (A):", mkonA, konAstd)
    print("Bootstrapping mean and std (B):", mkonB, konBstd)
    print("Bootstrapping MSMRD mean and std (A):", MSMRDmkonA, MSMRDkonAstd)
    print("Bootstrapping MSMRD mean and std (B):", MSMRDmkonB, MSMRDkonBstd)

In [ ]:
# Normalizing weights
weightsA = np.ones_like(fptsA)/float(len(fptsA))
weightsAMSMRD = np.ones_like(MSMRDfptsA)/float(len(MSMRDfptsA))
# Plot histograms
hbins = range(0, 600,15)
plt.hist(fptsA, bins = hbins, alpha=a1, color=c1, weights = weightsA );
plt.hist(fptsA, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step', 
         facecolor = 'None', weights = weightsA, label ='A');
plt.hist(MSMRDfptsA, bins = hbins, alpha=a1, color=c2, weights = weightsAMSMRD);
plt.hist(MSMRDfptsA, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step', 
         weights = weightsAMSMRD, label ='MSMRD-A');
plt.xlim([0,500])
plt.legend()

In [ ]:
# Normalizing weights
weightsB = np.ones_like(fptsB)/float(len(fptsB))
weightsBMSMRD = np.ones_like(MSMRDfptsB)/float(len(MSMRDfptsB))
# Plot histograms
hbins = range(0, 600,15)
plt.hist(fptsB, bins = hbins, alpha=a1, color=c1, weights = weightsB);
plt.hist(fptsB, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step',
         facecolor='None', weights = weightsB, label ='B',);
plt.hist(MSMRDfptsB, bins = hbins, alpha=a1, color=c2, weights = weightsBMSMRD);
plt.hist(MSMRDfptsB, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step',
         facecolor='None', weights = weightsBMSMRD, label ='MSMRD-B',);
plt.xlim([0,500])
plt.legend()

In [ ]:
# PLOT FOR PAPER #
# Configure figure
fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True, figsize=(6,5))
fig.subplots_adjust(hspace=0)
fsize = 16
plt.rcParams['xtick.labelsize']=fsize
plt.rcParams['ytick.labelsize']=fsize
# Set outer common axes titles
ax0 = fig.add_subplot(111, frameon=False)
ax0.set_xlabel("time", fontsize=fsize + 2)
ax0.set_ylabel("FPTs distribution", labelpad=17, fontsize=fsize + 2)
ax0.set_xticks([0])
ax0.set_yticks([0])
hbins = range(0, 500,15)

# Plot A to unbound histogram
ax1.hist(fptsA, bins = hbins, alpha=a1, color=c1, weights = weightsA );
ax1.hist(fptsA, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step', 
         facecolor = 'None', weights = weightsA, label ='Benchmark');
ax1.hist(MSMRDfptsA, bins = hbins, alpha=a1, color=c2, weights = weightsAMSMRD);
ax1.hist(MSMRDfptsA, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step', 
         weights = weightsAMSMRD, label ='MSM/RD');
ax1.legend(fontsize=fsize, title=r'Unbound $\rightarrow$ A ', title_fontsize = fsize, fancybox=True)
ax1.set_xlim([0,500])
ax1.set_ylim([0,0.25])


# Plot B to unbound histogram
ax2.hist(fptsB, bins = hbins, alpha=a1, color=c1, weights = weightsB);
ax2.hist(fptsB, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step',
         facecolor='None', weights = weightsB, label ='Benchmark',);
ax2.hist(MSMRDfptsB, bins = hbins, alpha=a1, color=c2, weights = weightsBMSMRD);
ax2.hist(MSMRDfptsB, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step',
         facecolor='None', weights = weightsBMSMRD, label ='MSM/RD',);
ax2.legend(fontsize=fsize, title=r' Unbound $\rightarrow$ B', title_fontsize = fsize, fancybox=True)
ax2.set_xlim([0,500])
ax2.set_ylim([0,0.25])
#plt.savefig('toBound_FPTs.pdf')

## Get bound2unbound FPT data of bechmark and MSMRD from files

In [ ]:
# Load benchmark FPT data computed by script
numTrajs = 10000
filenameA = '../../data/dimer/first_passage_times/patchyDimerFPTs_A2unbound_trajs' + \
str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
filenameB = '../../data/dimer/first_passage_times/patchyDimerFPTs_B2unbound_trajs' + \
str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
fptsA2unbound = []
fptsB2unbound = []
with open(filenameA) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        fptsA2unbound.append(time)
with open(filenameB) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        fptsB2unbound.append(time)

In [ ]:
# Load FPT data obtained with MSMRD integrator
lagtime = 150
numTrajs = 10000
filenameA = '../../data/dimer/first_passage_times/MSMRDpatchyDimerFPTs_A2unbound_trajs' + \
str(numTrajs) + '_lagt' + str(lagtime) +  '_boxsize' + str(boxsize) + '.xyz'
filenameB = '../../data/dimer/first_passage_times/MSMRDpatchyDimerFPTs_B2unbound_trajs' + \
str(numTrajs) + '_lagt' + str(lagtime) +  '_boxsize' + str(boxsize) + '.xyz'
MSMRDfptsA2unbound = []
MSMRDfptsB2unbound = []
with open(filenameA) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        MSMRDfptsA2unbound.append(time)
with open(filenameB) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        MSMRDfptsB2unbound.append(time)

## Plots of FPT data from benchmark and MSMRD simulations

In [ ]:
# Compuate mean and standard deviation of bootstrapped samples of benchmark
numBootsrapSamples = 200
mfptA2unbound, fptA2unboundstd = analysisTools.bootstrapping(fptsA2unbound, numBootsrapSamples)
mfptB2unbound, fptB2unboundstd = analysisTools.bootstrapping(fptsB2unbound, numBootsrapSamples)
print("Raw MFPTs (A and B): ", np.array(fptsA2unbound).mean(), np.array(fptsB2unbound).mean() )
print("Bootstrapping mean and std (A): ", mfptA2unbound, fptA2unboundstd)
print("Bootstrapping mean and std (B): ", mfptB2unbound, fptB2unboundstd)
MSMRDmfptA2unbound, MSMRDfptA2unboundstd = analysisTools.bootstrapping(MSMRDfptsA2unbound, numBootsrapSamples)
MSMRDmfptB2unbound, MSMRDfptB2unboundstd = analysisTools.bootstrapping(MSMRDfptsB2unbound, numBootsrapSamples)
print("Raw MSMRD MFPTs (A and B): ",np.array(MSMRDfptsA2unbound).mean(), np.array(MSMRDfptsB2unbound).mean())
print("MSMRD bootstrapping mean and std (A): ", MSMRDmfptA2unbound, MSMRDfptA2unboundstd)
print("MSMRD bootstrapping mean and std (B): ", MSMRDmfptB2unbound, MSMRDfptB2unboundstd)

In [ ]:
# Plotting histogram for A to unbound FPTs
# Normalizing weights
weightsA2 = np.ones_like(fptsA2unbound)/float(len(fptsA2unbound))
weightsAMSMRD2 = np.ones_like(MSMRDfptsA2unbound)/float(len(MSMRDfptsA2unbound))
# Plot histograms
hbins = np.arange(0, 6, 0.2)
plt.hist(fptsA2unbound, bins = hbins, alpha=a1, color=c1, weights = weightsA2);
plt.hist(fptsA2unbound, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step',
         facecolor='None', weights = weightsA2, label ='A');
plt.hist(MSMRDfptsA2unbound, bins = hbins, alpha=a1, color=c2, weights = weightsAMSMRD2);
plt.hist(MSMRDfptsA2unbound, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step',
         facecolor='None', weights = weightsAMSMRD2, label ='MSMRD-A');
plt.xticks(fontsize=16)
plt.yticks(np.arange(0, 0.16, step=0.04), fontsize=16)
plt.legend(fontsize=16)
plt.xlabel('time', fontsize=18)
plt.ylabel('FPTs distribution', fontsize=18)
plt.xlim([0,6])

In [ ]:
# Plotting histogram for B to unbound FPTs
# Normalizing weights
weightsB2 = np.ones_like(fptsB2unbound)/float(len(fptsB2unbound))
weightsBMSMRD2 = np.ones_like(MSMRDfptsB2unbound)/float(len(MSMRDfptsB2unbound))
# Plot histograms
hbins = np.arange(0, 6, 0.2)
plt.hist(fptsB2unbound, bins = hbins, alpha=a1, color=c1, weights = weightsB2);
plt.hist(fptsB2unbound, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step',
         facecolor='None', weights = weightsB2, label ='B');
plt.hist(MSMRDfptsB2unbound, bins = hbins, alpha=a1, color=c2, weights = weightsBMSMRD2);
plt.hist(MSMRDfptsB2unbound, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step',
         facecolor='None', weights = weightsBMSMRD2, label ='MSMRD-B');
plt.xticks(fontsize=16)
plt.yticks(np.arange(0, 0.16, step=0.04), fontsize=16)
plt.legend(fontsize=16)
plt.xlabel('time', fontsize=18)
plt.ylabel('FPTs distribution', fontsize=18)
plt.xlim([0,6])

In [ ]:
# PLOT FOR PAPER #
# Configure figure
fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True, figsize=(6,5))
fig.subplots_adjust(hspace=0)
fsize = 16
plt.rcParams['xtick.labelsize']=fsize
plt.rcParams['ytick.labelsize']=fsize
hbins = np.arange(0, 6, 0.2)

# Set outer common axes titles
ax0 = fig.add_subplot(111, frameon=False)
ax0.set_xlabel("time", fontsize=fsize + 2)
ax0.set_ylabel("FPTs distribution", labelpad=17, fontsize=fsize + 2)
ax0.set_xticks([0])
ax0.set_yticks([0])

# Plot A to unbound histogram
ax1.hist(fptsA2unbound, bins = hbins, alpha=a1, color=c1, weights = weightsA2);
ax1.hist(fptsA2unbound, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step',
         facecolor='None', weights = weightsA2, label = 'Benchmark');
ax1.hist(MSMRDfptsA2unbound, bins = hbins, alpha=a1, color=c2, weights = weightsAMSMRD2);
ax1.hist(MSMRDfptsA2unbound, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step',
         facecolor='None', weights = weightsAMSMRD2, label ='MSM/RD');
ax1.legend(fontsize=fsize, title=r'A $\rightarrow$ Unbound', title_fontsize = fsize, fancybox=True)
ax1.set_xlim([0,6])
ax1.set_ylim([0,0.15])
ax1.set_yticks(np.arange(0, 0.2, step=0.1));


# Plot B to unbound histogram
ax2.hist(fptsB2unbound, bins = hbins, alpha=a1, color=c1, weights = weightsB2);
ax2.hist(fptsB2unbound, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step',
         facecolor='None', weights = weightsB2, label ='Benchmark');
ax2.hist(MSMRDfptsB2unbound, bins = hbins, alpha=a1, color=c2, weights = weightsBMSMRD2);
ax2.hist(MSMRDfptsB2unbound, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step',
         facecolor='None', weights = weightsBMSMRD2, label ='MSM/RD');
ax2.legend(fontsize=fsize, title=r'B $\rightarrow$ Unbound', title_fontsize = fsize, fancybox=True)
ax2.set_xlim([0,6])
ax2.set_ylim([0,0.15])
ax2.set_yticks(np.arange(0, 0.2, step=0.1));
#plt.savefig('toUnbound_FPTs.pdf')

## Get bound2bound FPT data of bechmark and MSMRD from files (on trial)

In [ ]:
# Load benchmark FPT data computed by script
numTrajs = 10000
filenameA = '../../data/dimer/first_passage_times/patchyDimerFPTs_A2B_trajs' + \
str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
filenameB = '../../data/dimer/first_passage_times/patchyDimerFPTs_B2A_trajs' + \
str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
fptsA2B = []
fptsB2A = []
with open(filenameA) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        fptsA2B.append(time)
with open(filenameB) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        fptsB2A.append(time)

In [ ]:
# Load FPT data obtained with MSMRD integrator
lagtime = 150
numTrajs = 10000
filenameA = '../../data/dimer/first_passage_times/MSMRDpatchyDimerFPTs_A2B_trajs' + \
str(numTrajs) + '_lagt' + str(lagtime) +  '_boxsize' + str(boxsize) + '.xyz'
filenameB = '../../data/dimer/first_passage_times/MSMRDpatchyDimerFPTs_B2A_trajs' + \
str(numTrajs) + '_lagt' + str(lagtime) +  '_boxsize' + str(boxsize) + '.xyz'
MSMRDfptsA2B = []
MSMRDfptsB2A = []
with open(filenameA) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        MSMRDfptsA2B.append(time)
with open(filenameB) as file:
    for line in file:
        state, time = line.split(" ")
        time = float(time)
        MSMRDfptsB2A.append(time)

In [ ]:
# PLOT FOR PAPER #
# Configure figure
fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True, figsize=(6,5))
fig.subplots_adjust(hspace=0)
fsize = 16
plt.rcParams['xtick.labelsize']=fsize
plt.rcParams['ytick.labelsize']=fsize
hbins = np.arange(0, 500, 15)

# Set outer common axes titles
ax0 = fig.add_subplot(111, frameon=False)
ax0.set_xlabel("time", fontsize=fsize + 2)
ax0.set_ylabel("FPTs distribution", labelpad=17, fontsize=fsize + 2)
ax0.set_xticks([0])
ax0.set_yticks([0])

# Normalizing weights
weightsA2B = np.ones_like(fptsA2B)/float(len(fptsA2B))
weightsA2BMSMRD = np.ones_like(MSMRDfptsA2B)/float(len(MSMRDfptsA2B))
# Normalizing weights
weightsB2A = np.ones_like(fptsB2A)/float(len(fptsB2A))
weightsB2AMSMRD = np.ones_like(MSMRDfptsB2A)/float(len(MSMRDfptsB2A))

# Plot A to B histogram
ax1.hist(fptsA2B, bins = hbins, alpha=a1, color=c1, weights = weightsA2B);
ax1.hist(fptsA2B, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step',
         facecolor='None', weights = weightsA2B, label = 'Benchmark');
ax1.hist(MSMRDfptsA2B, bins = hbins, alpha=a1, color=c2, weights = weightsA2BMSMRD);
ax1.hist(MSMRDfptsA2B, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step',
         facecolor='None', weights = weightsA2BMSMRD, label ='MSM/RD');
ax1.legend(fontsize=fsize, title=r'A $\rightarrow$ B', title_fontsize = fsize, fancybox=True)
ax1.set_xlim([0,500])
ax1.set_ylim([0,0.2])
ax1.set_yticks(np.arange(0, 0.2, step=0.1));

# Plot B to A histogram
ax2.hist(fptsB2A, bins = hbins, alpha=a1, color=c1, weights = weightsB2A);
ax2.hist(fptsB2A, bins = hbins, alpha=a2, edgecolor=c1, lw=2, histtype='step',
         facecolor='None', weights = weightsB2A, label = 'Benchmark');
ax2.hist(MSMRDfptsB2A, bins = hbins, alpha=a1, color=c2, weights = weightsB2AMSMRD);
ax2.hist(MSMRDfptsB2A, bins = hbins, alpha=a2, edgecolor=c2, lw=2, histtype='step',
         facecolor='None', weights = weightsB2AMSMRD, label ='MSM/RD');
ax2.legend(fontsize=fsize, title=r'B $\rightarrow$ A', title_fontsize = fsize, fancybox=True)
ax2.set_xlim([0,500])
ax2.set_ylim([0,0.2])
ax2.set_yticks(np.arange(0, 0.2, step=0.1));

#plt.savefig('bound2bound_FPTs.pdf')

## Plot $K_{on}$ vs concentration plot

In [ ]:
# Load all data
boxsizes = [6, 7, 8, 9, 10]
numBoxes = len(boxsizes)
kon = [[] for i in range(numBoxes)]
konMSMRD = [[] for i in range(numBoxes)]
for i, boxsize in enumerate(boxsizes):
    filename1 = '../../data/dimer/first_passage_times/patchyDimerFPTs_2bound_trajs' + \
        str(numTrajs) + '_boxsize' + str(boxsize) + '.xyz'
    filename2 = '../../data/dimer/first_passage_times/MSMRDpatchyDimerFPTs_2bound_trajs' + \
        str(numTrajs) + '_lagt' + str(lagtime) +  '_boxsize' + str(boxsize) + '.xyz'
    with open(filename1) as file:
        for line in file:
            state, time = line.split(" ")
            kon[i].append(1.0/float(time))
    with open(filename2) as file:
        for line in file:
            state, time = line.split(" ")
            konMSMRD[i].append(1.0/float(time))

In [ ]:
# Calculate meand and bootstrapping error
numBootsrapSamples = 2000
meanKon = [0.0]*numBoxes
meanKonMSMRD = [0.0]*numBoxes
stdDevKon = [0.0]*numBoxes
stdDevKonMSMRD = [0.0]*numBoxes
for i, boxsize in enumerate(boxsizes):
    meanKon[i], stdDevKon[i] = analysisTools.bootstrapping(kon[i], numBootsrapSamples)
    meanKonMSMRD[i], stdDevKonMSMRD[i] = analysisTools.bootstrapping(konMSMRD[i], numBootsrapSamples)

In [ ]:
# Plot on rates vs concentration:
fig = plt.figure()
plt.rcParams['xtick.labelsize']=fsize
plt.rcParams['ytick.labelsize']=fsize
ax = fig.add_subplot(111)
linewidth = 0
fsize = 16
ax.errorbar(1.0/np.array(boxsizes), meanKon, yerr=stdDevKon, marker='o', markersize=7, elinewidth=2, 
            linewidth=linewidth , capsize=5, label='Benchmark')
ax.errorbar(1.0/np.array(boxsizes), meanKonMSMRD, yerr=stdDevKonMSMRD, marker='o', markersize=7, 
            elinewidth=2, linewidth=linewidth, capsize=5, label='MSM/RD')
#ax.set_yscale('log')
ax.set_xlabel(r'Concentration', fontsize = fsize+2)
ax.set_ylabel(r'$k_\mathrm{on}$', fontsize = fsize+8)
ax.set_ylim([0,0.12])
ax.legend(fontsize=fsize, title_fontsize = fsize, fancybox=True)

In [ ]:
np.array(meanKonMSMRD) - np.array(meanKon)

In [ ]:
1.0/np.array(boxsizes)